In [1]:
import pandas as pd
from IPython.display import Image
from scipy.constants import c
from kid_readout.analysis import resonator, plot_resonator

In [2]:
# Expectations
epsilon = 6.8
length = 30e-3
v = c / epsilon**(1/2)
wavelength = 2 * length
f = v / wavelength
1e-9 * f

1.916085993387465

In [3]:
def rename(df):
    df = df.rename(columns={'Frequency (GHz)': 'Frequency_GHz'})
    df = df.rename(columns=lambda column: column.translate(None, '[]'))
    return df

def load(df):
    df = rename(df)
    df['f'] = 1e9 * df.Frequency_GHz
    return df

In [4]:
s16 = load(pd.read_csv('/Users/daniel/Dropbox/multichroic/Sonnet/microstrip_resonator/coupler_16_um/standard_microstrip1_16_um.csv',
                       skiprows=range(7), delimiter=','))

In [7]:
s16

,Frequency_GHz,RES11,IMS11,RES12,IMS12,RES21,IMS21,RES22,IMS22,f
0,1.96,-0.000059,-0.002667,0.999812,-0.018745,0.999812,-0.018745,-0.000059,-0.002667,1.960000e+09
1,1.97,-0.000060,-0.002681,0.999810,-0.018842,0.999810,-0.018842,-0.000060,-0.002681,1.970000e+09
2,1.98,-0.000060,-0.002696,0.999808,-0.018939,0.999808,-0.018939,-0.000060,-0.002696,1.980000e+09


# Analyze

In [5]:
def mask_s21(f, s21, linewidth, n):
    return np.abs(f - f[np.argmin(np.abs(s21))]) < n * linewidth

def fit_resonator(df, linewidth, n=20):
    f = np.array(df.f)
    s21 = np.array(df.RES21 + 1j * df.IMS21)
    mask = mask_s21(f, s21, linewidth, n)
    r = resonator.Resonator(f[mask], s21[mask])
    return r, mask

In [6]:
res16, mask16 = fit_resonator(s16, 1e6, n=20)

ValueError: Parameter 'Q' has min == max

# Plot

In [ ]:
f = np.linspace(res16.freq_data.min(), res16.freq_data.max(), 1000)
scale = 1e-9
fig, ax = plt.subplots()
ax.set_xlabel('frequency [GHz]')
ax.set_ylabel('$|S_{21}|^2$ [dB]')
ax.plot(scale * res16.freq_data, 20 * np.log10(np.abs(res16.s21_data)), '-b', label='16 um')
ax.plot(scale * f, 20 * np.log10(np.abs(res16.model(x=f))), '-b')
ax.set_ylim(-0.05, 0.01)

In [ ]:
f = np.linspace(res16.freq_data.min(), res16.freq_data.max(), 1000)
s21_model = res16.model(x=f)
fig, ax = plt.subplots()
ax.set_xlabel('Re $S_{21}$')
ax.set_ylabel('Im $S_{21}$')
ax.plot(res16.s21_data.real, res16.s21_data.imag, '.b', label='16 um')
ax.plot(s21_model.real, s21_model.imag, '-b')
